In [1]:
# ------여기어때 호텔 리뷰 크롤링
# ------------------2024-08-26
# -----------------made by 서연

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pyperclip
import time
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

In [42]:
driver = webdriver.Chrome()

In [43]:
driver.get("https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&sortType=RECOMMEND&keyword=%EC%84%9C%EC%9A%B8&page=1&personal=2&checkIn=2024-08-26&checkOut=2024-08-27&category=2&grades=5%2C4")

In [15]:
soup  = bs(driver.page_source, 'lxml')

In [16]:
url_list = []
title_list = []
star_list = []

for page in range(1, 6):
    url = 'https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&sortType=RECOMMEND&keyword=%EC%84%9C%EC%9A%B8&page={}&personal=2&checkIn=2024-08-26&checkOut=2024-08-27&category=2&grades=5%2C4'.format(page)
    driver.get(url)

    time.sleep(1)

    soup = bs(driver.page_source, 'lxml')
    # 링크 가져오기
    urls = soup.select('a', class_='gc-thumbnail-type-seller-card.css-wels0m')

    for i in urls:
        url_list.append('https://www.yeogi.com' + i['href'])

    #호텔 이름 가져오기
    titles = soup.select('h3.gc-thumbnail-type-seller-card-title css-1geyv1d')
    
    for i in titles:
        title_list.append(i.text)

    # 호텔 성급 가져오기
    star = soup.select('ul.css-19akvy6')
        
    for i in star:
        star_list.append(i.text)

In [17]:
url_list

['https://www.yeogi.com/',
 'https://www.yeogi.com/domestic-accommodations/70163?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/6534?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/65229?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/51149?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/6264?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/69966?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/60449?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/64392?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.com/domestic-accommodations/57843?checkIn=2024-09-01&checkOut=2024-09-02&personal=2',
 'https://www.yeogi.c

In [18]:
urls_list = []
for i in url_list:
    if 'https://www.yeogi.com/domestic' in i:
        urls_list.append(i)

In [21]:
data = {"url" : urls_list}

In [22]:
len(urls_list), len(title_list), len(star_list)

(88, 0, 88)

In [23]:
yeogi_df = pd.DataFrame(data)

In [24]:
yeogi_df

,url
0,https://www.yeogi.com/domestic-accommodations/...
1,https://www.yeogi.com/domestic-accommodations/...
2,https://www.yeogi.com/domestic-accommodations/...
3,https://www.yeogi.com/domestic-accommodations/...
4,https://www.yeogi.com/domestic-accommodations/...
...,...
83,https://www.yeogi.com/domestic-accommodations/...
84,https://www.yeogi.com/domestic-accommodations/...
85,https://www.yeogi.com/domestic-accommodations/...
86,https://www.yeogi.com/domestic-accommodations/...


In [44]:
# 결과를 저장할 리스트
review_list = []

# 각 URL을 순회하여 리뷰 추출
for url in tqdm(urls_list):
    driver.get(url)
    time.sleep(0.5)  # 페이지 로드 대기

    soup = bs(driver.page_source, 'lxml')
    
    for _ in range(5):
        try:
            for page in range(3, 7):
                soup = bs(driver.page_source, 'lxml')
    
                review = soup.select('p.css-nyr29c')
            
                for i in review:
                    review_list.append(i.text)

                time.sleep(0.5)
                body = driver.find_element(By.CSS_SELECTOR, "body")

                time.sleep(0.5)
                body.send_keys(Keys.ENTER)
        
                time.sleep(0.5)
                driver.find_element(By.CSS_SELECTOR, 'div > button:nth-child({})'.format(page)).click()

                time.sleep(0.5)
                soup = bs(driver.page_source, 'lxml')
            
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="review"]/div[7]/div/div/button[7]').click()
            
        except:
            break

100%|█████████████████████████████████████████| 88/88 [1:05:15<00:00, 44.49s/it]


In [45]:
review_list

['⭐️ ”살면서 한번쯤 꼭 가봐야하는 호텔“⭐️\n\n호텔이름 : 신라호텔\n\n위치 : 서울 중구 동호로 249\n\n객실 : 더블룸 + 어반아일랜드 올데이 + 식음업장 할인권 5만원 pkg\n\n주차 :  신라호텔 내 주차장이 있어 쉽개 주차할 수 있습니다. 하지만 신라호텔 부지가 워낙 넓어 주차장에서 신라호텔 셔틀 버스를 타고 호텔로비로 이동해야합니다. 만약 제휴카드(아멕x 등)을 보유 하셨다면 발렛파킹을 무료로 입차 출차시 해주시니 참고하시길 마랍니다. 저는 발렛을 이용해 편하게 로비앞에서 바로 차를 맡겼습니다. 발렛 직원분들도 매우 친절했습니다.\n\n입실 : 로비에서 간단하게 체크인을 하고 얼른 객실부터 올라갔습니다. 체크인시 룸에 따라 디파짓(보증금)을 부담할 수 있으며 체크아웃시 환불됩니다. 체크인 시간 전이라도 언제든지 짐을 맡길 수 있으며 친절히 응대해주십니다.\n\n객실 : 정부지침으로 인해 일회용 어메니티는 제공되지 않지만 객실 내 다회용 삼푸 린스 바디워시 바디로션 비누 등은 제공되니 칫솔 치약 정도만 챙기면 됩니다. 객실내 유료 어메니티가 비치 더ㅣ어있으니 참고 하시길 바랍니다. 객실 내 냉장고에 다양한 음료, 술 등이 비치 되어있으나 가격이 사악합니다 역시 호텔 ㅎㅎ\n\n객실 컨디션 :  말해뭐해 최상급이었습니다. 객실 침구도 원하는 종류로 바꿀 수도 있었습니다. 객실 챡상 서랍에 객실 침구 종류들이 따로 적혀있으니 우너하는 침구로도 바꿔보시길 바랍니다. 저는 기본 침구도 너무 마음에 들어 그대로 사용했습니다. \n저녁에 1분만에 잠들고 체크아웃 직전에 일어날 만큼 매우 만족스러운 침구였습니다. \n\n객실 청결 : 객실 청결도 매우 깔끔하고 만족스러웠습니다. 객실 화장실도 머리카락 하나 없이 깨끗했고 화장실 내 위생봉투가 비치되어있으니 위생용품들을 버리실 때 참고하세요! \n욕조도 매우 깨끗했습니다.\n\n객실 기타 : 호텔 객실 티비들은 대부분 OTT가 되지 않는데 유튜브 넷플릭스 인터넷들을 티비로 바로 이용할 수 있어 매우 

In [46]:
len(review_list)

5963

In [47]:
region_list = []
for _ in range(len(review_list)):
    region_list.append("서울")

In [48]:
data = {"지역" : region_list, "리뷰" : review_list}

In [49]:
yeogi_seoul_df = pd.DataFrame(data)

In [50]:
# DataFrame을 피클 파일로 저장
df.to_pickle('여기어때_서울_호텔리뷰_서연.pkl')

NameError: name 'df' is not defined

In [ ]:
yeogi_seoul_df